In [34]:
import pandas as pd

In [35]:
data_path = r"C:\Users\hites\OneDrive\Desktop\Housing Regression\data\raw\HouseTS.csv"
df = pd.read_csv(data_path)

In [36]:
df.shape

(884092, 39)

In [37]:
df.head()

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,...,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full
0,2012-03-31,46550.0,217450.0,31.813674,110.183666,14.0,23.0,44.0,64.0,59.5,...,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,200773.999557,Atlanta-Sandy Springs-Alpharetta
1,2012-04-30,61870.0,245000.0,40.723982,130.528256,22.0,29.0,56.0,69.0,89.5,...,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,202421.064584,Atlanta-Sandy Springs-Alpharetta
2,2012-05-31,125500.0,217450.0,63.913043,119.919216,24.0,40.0,63.0,60.0,144.5,...,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,202681.309539,Atlanta-Sandy Springs-Alpharetta
3,2012-06-30,153000.0,189900.0,81.598080,105.617353,34.0,46.0,50.0,57.0,126.0,...,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,202998.603897,Atlanta-Sandy Springs-Alpharetta
4,2012-07-31,165500.0,154000.0,81.598080,83.921175,39.0,49.0,42.0,50.0,80.0,...,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,203781.903446,Atlanta-Sandy Springs-Alpharetta


In [38]:
df.columns

Index(['date', 'median_sale_price', 'median_list_price', 'median_ppsf',
       'median_list_ppsf', 'homes_sold', 'pending_sales', 'new_listings',
       'inventory', 'median_dom', 'avg_sale_to_list', 'sold_above_list',
       'off_market_in_two_weeks', 'city', 'zipcode', 'year', 'bank', 'bus',
       'hospital', 'mall', 'park', 'restaurant', 'school', 'station',
       'supermarket', 'Total Population', 'Median Age', 'Per Capita Income',
       'Total Families Below Poverty', 'Total Housing Units', 'Median Rent',
       'Median Home Value', 'Total Labor Force', 'Unemployed Population',
       'Total School Age Population', 'Total School Enrollment',
       'Median Commute Time', 'price', 'city_full'],
      dtype='object')

In [39]:
df['date'] = pd.to_datetime(df['date'])
print(df['date'].dt.year.value_counts())

date
2013    74712
2014    74712
2015    74712
2016    74712
2020    74712
2017    74712
2018    74712
2019    74712
2022    74712
2021    74712
2023    74712
2012    62260
Name: count, dtype: int64


### **Avoiding Leakage**

- Train set (2012-2019) is "clean" historical training.
- Eval set (2020-2021) is "unseen" during training, but still used for validation and hyperparameter tuning. 
- Holdout set (2022-2023) is completely untouched until the very end.

In [40]:
# Sorting by the date to avoid shuffling issues 
df = df.sort_values("date")  

In [41]:
# Cutoff dates 
cutoff_date_eval = "2020-01-01" # Validation starts 
cutoff_date_holdout = "2022-01-01" # Holdout starts

In [42]:
# Train before: 2020
train_df = df[df["date"] < cutoff_date_eval]

# Validation / Eval: 2020 - 2021
eval_df = df[(df["date"] >= cutoff_date_eval) & (df["date"] <= cutoff_date_holdout)]

# Holdout: 2022 - 2023
holdout_df = df[df["date"] >= cutoff_date_holdout]

In [43]:
print(f"Train Shape: {train_df.shape}")
print(f"Eval Shape: {eval_df.shape}")
print(f"Holdout Shape: {holdout_df.shape}")

Train Shape: (585244, 39)
Eval Shape: (149424, 39)
Holdout Shape: (149424, 39)


In [44]:
train_df.head(2)

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,...,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full
0,2012-03-31,46550.0,217450.0,31.813674,110.183666,14.0,23.0,44.0,64.0,59.5,...,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,200773.999557,Atlanta-Sandy Springs-Alpharetta
640562,2012-03-31,200000.0,7500.0,104.931794,79.265873,1.0,1.0,1.0,2.0,290.0,...,1108.0,641.0,94600.0,1171.0,52.0,2376.0,2376.0,1018.0,105863.681174,Pittsburgh


In [45]:
eval_df.head(2)

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,...,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full
137976,2020-01-31,184000.0,168450.0,123.333333,109.899800,35.0,30.0,34.0,37.0,50.0,...,4598.0,964.0,169000.0,5690.0,414.0,10573.0,10573.0,5089.0,164839.734849,Chicago-Naperville-Elgin
157714,2020-01-31,140000.0,139000.0,94.691957,111.311054,61.0,81.0,83.0,54.0,10.0,...,10164.0,802.0,132400.0,10479.0,552.0,19809.0,19809.0,9383.0,132164.314962,Cincinnati


In [46]:
holdout_df.head(2)

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,...,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full
587856,2022-01-31,340000.0,349999.0,189.867865,195.526649,282.0,287.0,241.0,73.0,12.0,...,22246.0,1289.0,261800.0,29485.0,1469.0,55621.0,55621.0,24381.0,314667.746005,Orlando-Kissimmee-Sanford
878956,2022-01-31,310000.0,269900.0,243.839836,239.543726,49.0,47.0,43.0,7.0,6.5,...,7865.0,1161.0,206600.0,7846.0,345.0,15423.0,15423.0,6733.0,325271.243813,Tampa-St. Petersburg-Clearwater


In [48]:
train_df.to_csv(r"C:\Users\hites\OneDrive\Desktop\Housing Regression\data\raw\train.csv", index = False)
eval_df.to_csv(r"C:\Users\hites\OneDrive\Desktop\Housing Regression\data\raw\eval.csv", index = False)
holdout_df.to_csv(r"C:\Users\hites\OneDrive\Desktop\Housing Regression\data\raw\holdout.csv", index = False)